In [1]:
import pandas as pd
import pyodbc


conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=LAPTOP-J35LO566\\SQLEXPRESS;'
    'DATABASE=DataWarehouseAnalytics;'
    'Trusted_Connection=yes;'
)


In [2]:
query ="""
WITH customer_orders AS (
    SELECT 
        c.customer_key,
        c.customer_number,
        c.country,
        CONCAT(c.first_name ,' ', c.last_name) AS customer_name,
        DATEDIFF(YEAR, c.birthdate, GETDATE()) AS customer_age,
        MIN(s.order_date) AS first_order_date,
        MAX(s.order_date) AS last_order_date,
        COUNT(DISTINCT s.order_number) AS total_orders,
        SUM(s.sales_amount) AS total_spending,
        SUM(s.quantity) AS total_quantity,
        COUNT(DISTINCT s.product_key) AS total_products
    FROM [gold.fact_sales] s 
    LEFT JOIN [gold.dim_customers] c ON s.customer_key = c.customer_key 
    GROUP BY 
        c.customer_key,
        c.customer_number,
        c.first_name,
        c.last_name,
        c.birthdate,
        c.country
),
customer_segments AS (
    SELECT 
        customer_key,
        customer_number,
        customer_name,
        country,
        customer_age,
        first_order_date,
        last_order_date,
        total_orders,
        total_spending,
        total_quantity,
        total_products,
        DATEDIFF(month, last_order_date, GETDATE()) AS recency_in_months,
        DATEDIFF(month, first_order_date, GETDATE()) AS Lifespan_in_months,
        CASE 
            WHEN DATEDIFF(month, first_order_date, GETDATE()) = 0 THEN 0 
            ELSE total_spending / DATEDIFF(month, first_order_date, GETDATE())
        END AS avg_monthly_revenue,
        CASE 
            WHEN total_orders = 0 THEN 0
            ELSE total_spending / total_orders
        END AS avg_order_value
    FROM customer_orders
)
SELECT 
    customer_key,
    customer_number,
    customer_name,
    country,
    customer_age,
    first_order_date,
    last_order_date,
    total_orders,
    total_spending,
    total_quantity,
    total_products,
    recency_in_months,
    Lifespan_in_months,
    avg_monthly_revenue,
    avg_order_value,
    CASE 
        WHEN customer_age < 20 THEN 'under_20'
        WHEN customer_age BETWEEN 21 AND 30 THEN '21_30'
        WHEN customer_age BETWEEN 31 AND 40 THEN '31_40'
        WHEN customer_age BETWEEN 41 AND 50 THEN '41_50'
        WHEN customer_age BETWEEN 51 AND 60 THEN '51_60'
        ELSE 'above_60'
    END AS age_group,
    CASE 
        WHEN recency_in_months <= 1 THEN 'Active_customer' 
        WHEN recency_in_months BETWEEN 2 AND 3 THEN 'medium_active_customer'
        ELSE 'low_active_customer'
    END AS customer_activity,
    CASE 
        WHEN total_spending > AVG(total_spending) OVER () THEN 'high_spender'
        WHEN total_spending = AVG(total_spending) OVER () THEN 'Normal_spender'
        ELSE 'below_AVG'
    END AS spending_segment
FROM customer_segments ; 


"""

In [6]:

df = pd.read_sql(query, conn)
df.head(50)

C:\Users\joo\AppData\Local\Temp\ipykernel_6060\1109215751.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


,customer_key,customer_number,customer_name,country,customer_age,first_order_date,last_order_date,total_orders,total_spending,total_quantity,total_products,recency_in_months,Lifespan_in_months,avg_monthly_revenue,avg_order_value,age_group,customer_activity,spending_segment
0,1,AW00011000,Jon Yang,Australia,54.0,2011-01-19,2013-05-03,3,8249,8,8,149.0,177.0,46.0,2749,51_60,low_active_customer,high_spender
1,2,AW00011001,Eugene Huang,Australia,49.0,2011-01-15,2013-12-10,3,6384,11,10,142.0,177.0,36.0,2128,41_50,low_active_customer,high_spender
2,3,AW00011002,Ruben Torres,Australia,54.0,2011-01-07,2013-02-23,3,8114,4,4,152.0,177.0,45.0,2704,51_60,low_active_customer,high_spender
3,4,AW00011003,Christy Zhu,Australia,52.0,2010-12-29,2013-05-10,3,8139,9,9,149.0,178.0,45.0,2713,51_60,low_active_customer,high_spender
4,5,AW00011004,Elizabeth Johnson,Australia,46.0,2011-01-23,2013-05-01,3,8196,6,6,149.0,177.0,46.0,2732,41_50,low_active_customer,high_spender
5,6,AW00011005,Julio Ruiz,Australia,49.0,2010-12-30,2013-05-02,3,8121,6,6,149.0,178.0,45.0,2707,41_50,low_active_customer,high_spender
6,7,AW00011006,Janet Alvarez,Australia,49.0,2011-01-24,2013-05-14,3,8119,5,5,149.0,177.0,45.0,2706,41_50,low_active_customer,high_spender
7,8,AW00011007,Marco Mehta,Australia,56.0,2011-01-09,2013-03-19,3,8211,8,8,151.0,177.0,46.0,2737,51_60,low_active_customer,high_spender
8,9,AW00011008,Rob Verhoff,Australia,50.0,2011-01-25,2013-03-02,3,8106,7,7,151.0,177.0,45.0,2702,41_50,low_active_customer,high_spender
9,10,AW00011009,Shannon Carlson,Australia,56.0,2011-01-27,2013-05-09,3,8091,5,5,149.0,177.0,45.0,2697,51_60,low_active_customer,high_spender
